# Lineární klasifikace obrázků CIFAR10

Úkolem cvičení je naprogramovat lineární klasifikátor, který bude rozpoznávat objekty z datasetu CIFAR-10.
Vstupem je RGB obrázek o rozměrech 32$\times$32 pixelů a úkolem říci, který z 10 možných objektů (tříd) je na něm zachycen. Možnosti jsou tyto: letadlo, automobil, pták, kočka, jelen, pes, žába, kůň, loď, náklaďák.

**Model**
- Vstup: $x$, rozměr $N \times D$, kde $N$ je počet obrázků a $D$ počet číselných hodnot v jednom obrázku
  - Každý řádek je jeden obrázek reprezentovaný jako vektor, čísla v rozmezí 0-1.
- Parametr váhy (weights): $W$, rozměr $D \times C$
- Parametr bias: $b$, rozměr $C$
- Predikované skóre (logity): $s = xW + b$, rozměr $N \times C$
  - Pro každý obrázek predikce $C$ hodnot
- Predikovaná třída: $z = \arg\max_c s$, rozměr $N$
  - Pro každý obrázek vybereme třídu s nejvyšším skóre.

**Optimalizace**
- Kritérium křížová entropie: $L = \sum_c{p_c \cdot \log q_c}$
- Metoda Stochastic Gradient Descent (SGD)
  - $W := W - \gamma \frac{dL}{dW}$
  - $b := b - \gamma \frac{dL}{dW}$
  - Hyperparametr $\gamma$ značí rychlost učení (learning rate)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')  # import tests
from typing import Callable

import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision

import ans
from tests import test_linear_classification

In [ ]:
torch.set_printoptions(profile='short')

# Načtení dat

Balík torchvision podporuje některé znamé datasety, mezi něž patří i CIFAR-10. Nemusíme tedy data stahovat z internetu manuálně, torchvision za nás vše obstará automaticky. Data uložíme do adresáře `./data`. Všimněme si flagu `train=True`, který říká, že se má načíst trénovací množina datasetu CIFAR-10 (soubory `data_batch_*`).

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True)
train_dataset

Validovat budeme na zbylých 10000 obrázcích. Získáme je nastavením `train=False`.

In [ ]:
val_dataset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True)
val_dataset

Seznam tříd je uložen v atributu `.classes`.

In [ ]:
train_dataset.classes

Objekt třídy `torchvision.datasets.cifar.CIFAR10` se chová podobně jako pythonovský `list` s tím, že každý jeho prvek je dvojice (obrázek, třída).

In [ ]:
train_dataset[5]

Jak vidíme, 6. prvek datasetu je *dvojice* (`tuple`) sestávající z obrázku a jeho indexu třídy (label, target).

- Obrázek je defaultně navrácen jako typ `Image` knihovny Pillow (Python Imaging Library, PIL).

- Target je číslo typu `int` označující jednu ze tříd v atributu `.classes`.

Všechny obrázky CIFAR-10 datasetu jsou uloženy v atributu `.data`, což je 4D `numpy.ndarray` typu `np.uint8`. První dimenze odpovídá jednotlivým obrázkům, další pak řádkům, sloupcům a kanálům (RGB).

In [ ]:
type(train_dataset.data), train_dataset.data.shape, train_dataset.data.dtype, train_dataset.data.min(), train_dataset.data.max()

Podobně všechny targety jsou uložny v `.targets`, což je `list` o délce počtu obrázků, přičemž každý target číslo typu `int` v rozmezí 0-9, kde 0 značí letadlo (airplane), 1 značí automobil (automobile) atd.

In [ ]:
type(train_dataset.targets), len(train_dataset.targets), type(train_dataset.targets[0]), min(train_dataset.targets), max(train_dataset.targets)

Pro lepší představu si vykreslíme sloupec 10 obrázků pro každou z 10 tříd.

In [ ]:
plt.figure(figsize=(12, 12))

for i, cls in enumerate(train_dataset.classes):
    # chceme pouze obrazky aktualni tridy a z nich nahodne vybereme 10
    cls_ids = [j for j, y in enumerate(train_dataset.targets) if y == i]
    draw_ids = np.random.choice(cls_ids, size=10)
    
    # pyplot podobne jako MATLAB nabizi funkci subplot pro vykresleni vice grafu do jednoho okna
    for j, k in enumerate(draw_ids):
        # vykresli 10x10 obrazku, poradi je po radcich, ovsem my budeme vykreslovat po sloupcich,
        # tj. kazdy sloupec bude obsahovat 10 prikladu jedne ze trid
        plt.subplot(10, 10, j * 10 + i + 1)
        
        # vyresli obrazek
        plt.imshow(train_dataset.data[k])
        
        # nevykresluj popisky os
        plt.axis('off')
        
        # v prvnim radku pridame nazev grafu (obrazku)
        if j == 0:
            plt.title(cls, fontsize=10)
plt.show()

# Načítání dat po dávkách

Prvním úkolem bude implementovat načítání dat po dávkách, na kterých se bude klasifikátor učit. Funkcionalita je implementovaná třídou `ans.data.BatchLoader` a nachází se v souboru `ans/data.py`. Třída implementuje metodu `__iter__` a je tedy možné ji použít jako zdroj např. pro `for` cyklus následujícím způsobem:
``` python
train_loader = ans.data.BatchLoader(
    torch.tensor(train_dataset.data),  # input images, shape (50000, 32, 32, 3)
    torch.tensor(train_dataset.targets),  # targets, shape (50000,)
    batch_size=100,
    shuffle=True  # return the data in random order
)
for inputs, targets in train_loader:
    # inputs ... shape (100, 32, 32, 3)
    # targets ... shape (100,)
    ...
```
**Úkol: implementujte metodu `__iter__` třídy `ans.data.BatchLoader`.**

Implementaci lze zkontrolovat připravenými unit testy.

In [ ]:
test_linear_classification.TestBatchLoader.eval()

In [ ]:
train_loader = ans.data.BatchLoader(
    torch.tensor(train_dataset.data),
    torch.tensor(train_dataset.targets),
    batch_size=5,
    shuffle=True
)
train_loader

In [ ]:
inputs, targets = next(iter(train_loader))
print(type(inputs), inputs.shape, inputs.dtype, inputs.min(), inputs.max())
print(type(targets), targets.shape, targets.dtype, targets.min(), targets.max())

In [ ]:
val_loader = ans.data.BatchLoader(
    torch.tensor(val_dataset.data),
    torch.tensor(val_dataset.targets),
    batch_size=5,
    shuffle=False
)
val_loader

# Preprocessing

Data jsou celočiselného typu `torch.uint8`, což jsou čísla v rozmezí 0-255. Pro lepší numerické vlastnosti data převedeme do rozsahu 0-1 a datového typu `torch.float32`, který je pro PyTorch výchozí. Operaci provedeme jednoduchým vydělením max. hodnotou 255. Zároveň obrázky přetvarujeme do vektoru, takže výstup bude mít rozměr (batch_size, počet_pixelů). Celé předzpracování bude implementovat funkce `preprocess`, kterou budeme volat pro každou dávku po jejím načtení z `BatchLoader`u. Normalizovat budeme pouze obrázky, s targety nic dělat nebudeme.

**Úkol: implementuje funkci `preprocess`.**

In [ ]:
def preprocess(inputs: torch.Tensor) -> torch.Tensor:
    """
    Args:
        inputs: n-dimensional tensor with first dimension of size num_inputs
    Returns:
        outputs: 2-dimensional tensor; shape (num_inputs, num_features), dtype float32
    """
    
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################
    
    return outputs

In [ ]:
test_linear_classification.TestPreprocess.eval(preprocess_fn=preprocess)

In [ ]:
inputs_prep = preprocess(inputs)
inputs_prep

# Inicializace

Váhovou matici $W$ klasifikátoru inicializujeme na náhodné hodnoty s normálním rozdělením a malou standardní odchylkou. Bias inicializujeme vektor nul s odpovídajícím rozměrem. Inicializaci bude provádět funkce `init_params`, která převezme rozměry a požadovanou odchylku (parametr `multiplier`) a vrátí dvojici (weight, bias).

Pozn.: nezapomínejme, že obrázky jsou uloženy *po řádcích* a náš model má podobu $s = xW + b$. Od toho se odvíjejí rozměry $W$ a $b$.

**Úkol: implementuje funkci `init_params`.**

In [ ]:
def init_params(input_dim: int, output_dim: int, multiplier: float = 1e-2) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Args:
        input_dim: size of one input
        output_dim: size of one output
        multiplier: standard deviation of weight
    Returns:
        weight, bias
    """
    
    ########################################
    # TODO: implement
    
    

    # ENDTODO
    ########################################
    
    return weight, bias

In [ ]:
test_linear_classification.TestInit.eval(init_params_fn=init_params)

In [ ]:
weight, bias = init_params(...)
weight, bias

# Výpočet lineárních skóre (logitů)

Nyní vypočteme lineární skóre (logity) jako
$$s_n = x_nW + b$$
kde
- $s_n$ je vektor skóre pro $n$-tý vzorek (obrázek) $x_n$ a každou z $C$ tříd
- $x_n$ je $n$-tý vzorek (obrázek) v dávce reprezentovaný jako (řádkový) vektor
- $W$ je matice vah klasifikátoru
- $b$ je bias klasifikátoru

Výpočet bude zajišťovat funkce `calc_linear_scores`

**Úkol: implementuje funkci `calc_linear_scores`.**

In [ ]:
def calc_linear_scores(inputs: torch.Tensor, weight: torch.Tensor, bias: torch.Tensor) -> torch.Tensor:
    """
    Args:
        inputs: shape (num_samples, num_features)
        weight: shape (num_features, num_classes)
        bias: shape (num_classes,)
    Returns:
        scores: shape (num_samples, num_classes)
    """
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################
    
    return scores

In [ ]:
test_linear_classification.TestCalcLinearScores.eval(calc_linear_scores_fn=calc_linear_scores)

In [ ]:
logits = calc_linear_scores(inputs_prep, weight, bias)
logits

# Kritérium: softmax cross entropy loss

Výstupní pravděpodobnost $c$-té třídy $q_{nc}$ predikovaná klasifikátorem pro $x_n$ získáme aplikací funkce softmax na skóre $s$
$$q_{nc} = \frac{\exp{s_{nc}}}{\sum_{i=1}^{C}{\exp{s_{ni}}}}, c=1,\ldots,C$$
kde
- $s_{nj}$ je predikované skóre $n$-tého vzorku dávky třídy $j$

Klasifikátor budeme trénovat optimalizací křížové entropie mezi predikovanými $q_{nc}$ a skutečnými pravděpodobnostmi $p_{nc}$
$$L_n = \sum_c{p_{nc} \log q_{nc}}$$
kde
- $L_n$ je hodnota kritéria pro $n$-tý vzorek $x_n$ dávky $x$

Jelikož pro každý obrázek známe, který objekt je na něm zachycen, skutečné diskrétní rozdělení pravděpodobnosti $p_n=[p_{n1},\ldots,p_{nC}]$ má pouze jednu hodnotu $p_{nc}=1$ a to správnou třídu $c=y_n$; ostatní $p_{nc}=0, c \ne y_n$ jsou nulové. Jde tedy o tzv. one hot vektor. Sloučením softmaxu a odstranění nulových členů ve vztahu pro výpočet entropie získáme
$$L_n = \log q_{ny_n} = \log\frac{\exp{s_{ny_n}}}{\sum_{c=1}^{C}{\exp{s_{nc}}}} = -s_{ny_n} + \log\sum_c\exp s_{nc}$$
Loss tedy můžeme počítat přímo z lineárních skóre (logitů), což s sebou zároveň přináší výhodu modularity, protože později takto bude možné kritérium jednoduše vyměnit např. za hinge loss a namísto logistické regrese tak trénovat klasifikátor SVM. Kritérium bude počítat funkce `softmax_cross_entropy`.

**Úkol: implementujte funkci `softmax_cross_entropy`**

In [ ]:
def softmax_cross_entropy(scores: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    """
    Args:
        scores: output linear scores (logits before softmax); shape (num_samples, num_classes)
        targets: vector of class indicies (integers); shape (num_samples,)
    Returns:
        loss: averare cross entropy on the batch; tensor containing single number (scalar), e.g. "tensor(2.23)"
    """
    
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################
    
    return loss

In [ ]:
test_linear_classification.TestSoftmaxCrossEntropy.eval(softmax_cross_entropy_fn=softmax_cross_entropy)

In [ ]:
loss = softmax_cross_entropy(logits, targets)
loss

# Úspěšnost: accuracy

Kromě lossu budeme pro lepší orientaci měřit i přesnost (accuracy), byť tuto veličinu nebudeme přímo optimalizovat.

**Úkol: implementujte funkci `accuracy`**

In [ ]:
def accuracy(scores: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    """
    Args:
        scores: output linear scores (logits before softmax); shape (num_samples, num_classes)
        targets: vector of class indicies (integers); shape (num_samples,)
    Returns:
        acc: averare accuracy on the batch; tensor containing single number (scalar), e.g. "tensor(0.364)"
    """
    
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################
    
    return acc

In [ ]:
test_linear_classification.TestAccuracy.eval(accuracy_fn=accuracy)

# Gradient na parametry

Optimalizace lossu $L$ bude probíhat stochastickou metodou nejvějtšího spádu (Stochastic Gradient Descent, SGD). K tomu potřebujeme znát gradient na jednotlivé parametry, na kterých $L$ závisí:

Gradient na $c$-tý řádek matice $W$:
$$\frac{\partial L_n}{\partial w_c} = \frac{1}{N}(q_{nc} - p_{nc})x_n$$
kde
- $x_n$ $n$-tý vzorek dávky
- $q_{nc}$ je pravděpodobnost $c$-té třídy predikovaná pro $x_n$
- $p_{nc}$ je cílová požadovaná pravděpodobnost pro $x_n$
- $N$ je počet vzorků $x_n$ v dávce $x$.

Gradient na $c$-tý prvek biasu:
$$\frac{\partial L}{\partial b_c} = \frac{1}{N}(q_{nc} - p_{nc})$$

Oba gradienty bude vracet funkce `softmax_cross_entropy_gradients`, přičemž vstupem jí budou potřebné proměnné, tj. $x$, $q$ a $p$.

**Úkol: implementuje funkci `softmax_cross_entropy_gradients`**

In [ ]:
def softmax_cross_entropy_gradients(
    inputs: torch.Tensor,
    logits: torch.Tensor,
    targets: torch.Tensor
) -> tuple[torch.Tensor, torch.Tensor]:
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################
    
    return dweight, dbias

In [ ]:
test_linear_classification.TestSoftmaxCrossEntropyGradients.eval(softmax_cross_entropy_gradients_fn=softmax_cross_entropy_gradients)

In [ ]:
dweight, dbias = softmax_cross_entropy_gradients(inputs_prep, logits, targets)
dweight, dbias

# Update parametrů

Update bude probíhat metodou největšího spádu (Gradient Descent), tj. od aktuáního odhadu parametrů $\theta$ odečteme gradient $\frac{dL}{\theta}$ přeškálovaný krokem $\gamma$:

$\theta := \theta - \gamma \frac{dL}{\theta}$

Update implementujeme jako funkci, která převezme parametr, gradient na něj a krok učení $\gamma$ a parametr updatuje. Nebude přitom bytečně vytvářet jeho kopii, vše proběhne modifikací původního tensoru, anglicky tzv. inplace.

**Úkol: implementuje funkci `update_param_inplace`**

In [ ]:
def update_param_inplace(param: torch.Tensor, dparam: torch.Tensor, learning_rate: float) -> None:
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################

In [ ]:
test_linear_classification.TestUpdateParamInplace.eval(update_param_inplace_fn=update_param_inplace)

In [ ]:
weight, bias

In [ ]:
update_param_inplace(weight, dweight, 0.1)
update_param_inplace(bias, dbias, 0.1)

In [ ]:
weight, bias

# Spojení všech kroků dohromady: funkce `train_step` a `val_step`

Nyní spojíme všechny kroky do jedné funkce `train_step_softmax`, která převezme dávku vzorků, aktuální parametry klasifikátoru a hyperparametr krok učení a provede jeden krok směrem k mnimializaci křížové entropie na dávce. Funkce vrátí hodnotu lossu a přesnosti (accuracy) dosaženou na dávce.

**Úkol: implementuje funkci `train_step_softmax`**

In [ ]:
def train_step_softmax(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    weight: torch.Tensor,
    bias: torch.Tensor,
    learning_rate: float = 1e-3
) -> tuple[float, float]:
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################
    
    return loss.item(), acc.item()

In [ ]:
test_linear_classification.TestTrainStepSoftmax.eval(train_step_softmax_fn=train_step_softmax)

**Úkol: implementuje funkci `val_step`**

In [ ]:
def val_step(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    weight: torch.Tensor,
    bias: torch.Tensor,
    loss_fn: Callable[[torch.Tensor, torch.Tensor], tuple[float, float]]  # e.g. softmax_cross_entropy
) -> tuple[float, float]:
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################
    
    return loss.item(), acc.item()

In [ ]:
test_linear_classification.TestValStep.eval(val_step_fn=val_step, loss_fn=softmax_cross_entropy)

In [ ]:
def validate(
    loader: ans.data.BatchLoader,
    weight: torch.Tensor,
    bias: torch.Tensor,
    loss_fn: Callable[[torch.Tensor, torch.Tensor], tuple[float, float]]  # e.g. softmax_cross_entropy
) -> tuple[float, float]:
    total_loss = 0.
    total_acc = 0.
    for inputs, targets in loader:
        loss, acc = val_step(inputs, targets, weight, bias, loss_fn)
        total_loss += loss
        total_acc += acc
    return total_loss / len(loader), total_acc / len(loader)

In [ ]:
ans.utils.seed_everything(0)

# hyperparameters
num_epochs = ...
learning_rate = ...

# data loaders
train_loader = ans.data.BatchLoader(
    torch.tensor(train_dataset.data),
    torch.tensor(train_dataset.targets),
    batch_size=...,
    shuffle=True
)
val_loader = ans.data.BatchLoader(
    torch.tensor(val_dataset.data),
    torch.tensor(val_dataset.targets),
    batch_size=...,
    shuffle=False
)

# init parameters
weight, bias = init_params(...)

# validate once before training
train_loss, train_acc = validate(train_loader, weight, bias, softmax_cross_entropy)
val_loss, val_acc = validate(val_loader, weight, bias, softmax_cross_entropy)
print(f"after init: train_loss={train_loss:.5f}, train_acc={train_acc:.3f}, val_loss={val_loss:.5f}, val_acc={val_acc:.3f}")

# optimize
for epoch in range(num_epochs):
    # train loop
    for inputs, targets in train_loader:
        loss, acc = train_step_softmax(inputs, targets, weight, bias, learning_rate=learning_rate)
        train_loss = 0.99 * train_loss + 0.01 * loss
        train_acc = 0.99 * train_acc + 0.01 * acc
    
    # validation loop
    # train_loss, train_acc = validate(train_loader, weight, bias, softmax_cross_entropy)
    val_loss, val_acc = validate(val_loader, weight, bias, softmax_cross_entropy)
    
    # print
    print(f"epoch {epoch + 1}: train_loss={train_loss:.5f}, train_acc={train_acc:.3f}, val_loss={val_loss:.5f}, val_acc={val_acc:.3f}")

# Support Vector Machine (SVM)

SVM je softmaxu velmi podobné. Z pohledu neuronových sítí se liší pouze způsobem výpočtu lossu - místo cross entropy použijeme hinge loss:
$$L_n = \sum_{c\ne y_n}\max(0, 1 + s_{nc} - s_{ny})$$
kde:
- $y_n$ je správný index třídy na vzorku (obrázku) $x_n$
- $s_{ni}$ je skóre (logity) predikované lineárním klasifikátorem pro $n$-tý obrázek a $i$-tou třídu.

Gradient na váhy pak je
$$\frac{\partial L_n}{\partial w_{y_n}} = -\sum_{c\ne y_n}\boldsymbol{1}(1 + s_{nc} - s_{ny} > 0)x_n$$
$$\frac{\partial L_n}{\partial w_{c\ne y_n}} = \boldsymbol{1}(1 + s_c - s_y > 0)x_n$$
a pro biasy podobně, pouze bez násobení $x_n$ (na konci).

**Úkol: implementuje funkce `hinge_loss`, `hinge_loss_gradients` a `train_step_svm`**

In [ ]:
def hinge_loss(scores: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    """
    Args:
        scores: output from linear classifier, i.e. the pre-softmax logits; shape (num_samples, num_classes)
        targets: vector of class indicies (integers); shape (num_samples,)
    Returns:
        loss: averare Weston-Watkins hinge loss on the batch; tensor containing single number (scalar), e.g. "tensor(2.374)"
    """
    
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################
    
    return loss

In [ ]:
test_linear_classification.TestHingeLoss.eval(hinge_loss_fn=hinge_loss)

In [ ]:
def hinge_loss_gradients(
    inputs: torch.Tensor,
    scores: torch.Tensor,
    targets: torch.Tensor
) -> tuple[torch.Tensor, torch.Tensor]:
    
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################
    
    return dweight, dbias

In [ ]:
test_linear_classification.TestHingeLossGradients.eval(hinge_loss_gradients_fn=hinge_loss_gradients)

In [ ]:
def train_step_svm(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    weight: torch.Tensor,
    bias: torch.Tensor,
    learning_rate: float = 1e-3
) -> tuple[float, float]:
    ########################################
    # TODO: implement
    
    
    
    # ENDTODO
    ########################################
    
    return loss.item(), acc.item()

In [ ]:
test_linear_classification.TestTrainStepSVM.eval(train_step_svm_fn=train_step_svm)

In [ ]:
ans.utils.seed_everything(0)

# hyperparameters
num_epochs = ...
learning_rate = ...

# data loaders
train_loader = ans.data.BatchLoader(
    torch.tensor(train_dataset.data),
    torch.tensor(train_dataset.targets),
    batch_size=...,
    shuffle=True
)
val_loader = ans.data.BatchLoader(
    torch.tensor(val_dataset.data),
    torch.tensor(val_dataset.targets),
    batch_size=...,
    shuffle=False
)

# init parameters
weight, bias = init_params(...)

# validate once before training
train_loss, train_acc = validate(train_loader, weight, bias, hinge_loss)
val_loss, val_acc = validate(val_loader, weight, bias, hinge_loss)
print(f"after init: train_loss={train_loss:.5f}, train_acc={train_acc:.3f}, val_loss={val_loss:.5f}, val_acc={val_acc:.3f}")

# optimize
for epoch in range(num_epochs):
    # train loop
    for inputs, targets in train_loader:
        loss, acc = train_step_svm(inputs, targets, weight, bias, learning_rate=learning_rate)
        train_loss = 0.99 * train_loss + 0.01 * loss
        train_acc = 0.99 * train_acc + 0.01 * acc
    
    # validation loop
    # train_loss, train_acc = validate(train_loader, weight, bias, hinge_loss)
    val_loss, val_acc = validate(val_loader, weight, bias, hinge_loss)
    
    # print
    print(f"epoch {epoch + 1}: train_loss={train_loss:.5f}, train_acc={train_acc:.3f}, val_loss={val_loss:.5f}, val_acc={val_acc:.3f}")